Load the API keys from the `.env` file in the root directory.

In [25]:
from dotenv import load_dotenv, find_dotenv

if find_dotenv():
    load_dotenv()

Configure the LLM model.

In [26]:
LLM_NAME = "Mistral-7B-Instruct-v0.2"
# LLM_NAME = "Mixtral-8x7B-Instruct-v0.1"
# LLM_NAME = "gpt-3.5-turbo"

[Blablador](https://helmholtz-blablador.fz-juelich.de/) uses different names, so here, we create a quick mapping of the supported models.

In [27]:
BLABLADOR_MODEL_NAMES = {
    "Mistral-7B-Instruct-v0.2": "1 - Mistral-7B-Instruct-v0.2 - the best option in general - fast and good",
    "Mixtral-8x7B-Instruct-v0.1": "2 - Mixtral-8x7B-Instruct-v0.1 Slower with higher quality",
    "starcoder2-15b": "3 - starcoder2-15b - A model for programming",
    "GritLM-7B": "5 - GritLM-7B - For Text-Embeddings",
}

In [28]:
from os import environ
from dspy import OpenAI, settings as dspy_settings



llm = OpenAI(
    model=(
        BLABLADOR_MODEL_NAMES[LLM_NAME]
        if LLM_NAME in BLABLADOR_MODEL_NAMES.keys()
        else LLM_NAME),
    api_key=environ["OPENAI_API_KEY"],
    api_base=environ.get("OPENAI_API_BASE"),
    temperature=0,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n=1,
)
dspy_settings.configure(lm=llm)
llm.kwargs

{'temperature': 0,
 'max_tokens': 250,
 'top_p': 1,
 'frequency_penalty': 0,
 'presence_penalty': 0,
 'n': 1,
 'model': '1 - Mistral-7B-Instruct-v0.2 - the best option in general - fast and good'}

Test the LLM with a basic prompt.

In [29]:
llm.basic_request(prompt="A quick brown fox jumps")

{'id': 'cmpl-3XM99r5xwT3n9niRgEKdvj',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'text': ' over a lazy dog.\n\nThis sentence is known as a pangram, a sentence that uses every letter of the alphabet at least once. It\'s a fun little language trick that can help you practice your typing skills or just impress your friends.\n\nBut where did this particular pangram come from? The origin of the sentence is a bit murky, but it\'s believed to have first appeared in print in the early 20th century. One popular theory is that it was created by a typesetter named Peter Mark Roget, who is also known for compiling the famous thesaurus.\n\nAccording to this theory, Roget was working on a typesetting manual and wanted to include a pangram as an example. He came up with the sentence "A quick brown fox jumps over a lazy dog" because it used every letter of the alphabet and flowed well when typed.\n\nAnother theory is that the sentence was inspired by a line from Edwa

In [30]:
from dataclasses import dataclass
from dspy import Module, Predict
from dspy.primitives.prediction import Prediction

@dataclass(frozen=True)
class AnswerPredict(Module):
    predict: Predict = Predict(signature="question -> answer")
    
    def forward(self, question: str) -> Prediction:
        # Here, you can do more fancy stuff, e.g., retrieve some relevant documents.
        return self.predict(question=question)

In [31]:
answer_predict = AnswerPredict()
answer_predict(question="How does Ibuprofen work to treat headaches?")

Prediction(
    answer='Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID) that works by reducing the production of prostaglandins, which are hormone-like substances that cause pain and inflammation in the body. In the case of a headache, Ibuprofen helps to relieve the pain by reducing inflammation in the blood vessels surrounding the brain. Additionally, Ibuprofen also acts as a analgesic, which means it directly blocks the transmission of pain signals to the brain. By reducing inflammation and blocking pain signals, Ibuprofen effectively treats headaches and provides relief.'
)

In [44]:
from dataclasses import dataclass
from dspy import Module, ChainOfThought, Signature, InputField, OutputField
from dspy.primitives.prediction import Prediction

class QuestionAnswer(Signature):
    """Given a health question, generate a concise answer."""
    question: str = InputField()
    context: str = InputField(desc="""A short text passage that is relevant to the question and should be cited in the answer.""")
    answer: str = OutputField(desc="A short answer to the question that must use the provided context.")
    enough_context: str = OutputField(desc="'yes' if the context provides enough relevant information to answer the question or 'no' otherwise.")



@dataclass(frozen=True)
class AnswerChainOfThought(Module):
    # Now, we use chain-of-thougt prompting instead of zero-shot or few-shot-prompting.
    chain_of_thought: ChainOfThought = ChainOfThought(
        signature=QuestionAnswer
    )

    def forward(self, question: str, context: str) -> Prediction:
        print("History")
        for item in llm.history:
            print("\t" + str(item))
        res = self.chain_of_thought(question=question, context=context)
        print("History")
        for item in llm.history:
            print("\t" + str(item))
        return res

In [45]:
answer_chain_of_thought = AnswerChainOfThought()
answer_chain_of_thought(
    question="How does Ibuprofen work to treat headaches?",
    # context="Ipuprofen inhibits the production of pain hormones.",
    context="Ipuprofen can be dangerous.",
)

History
	{'prompt': 'A quick brown fox jumps', 'response': {'id': 'cmpl-3XM99r5xwT3n9niRgEKdvj', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'text': ' over a lazy dog.\n\nThis sentence is known as a pangram, a sentence that uses every letter of the alphabet at least once. It\'s a fun little language trick that can help you practice your typing skills or just impress your friends.\n\nBut where did this particular pangram come from? The origin of the sentence is a bit murky, but it\'s believed to have first appeared in print in the early 20th century. One popular theory is that it was created by a typesetter named Peter Mark Roget, who is also known for compiling the famous thesaurus.\n\nAccording to this theory, Roget was working on a typesetting manual and wanted to include a pangram as an example. He came up with the sentence "A quick brown fox jumps over a lazy dog" because it used every letter of the alphabet and flowed well when typed.\n\nAnother theory is t

Prediction(
    rationale='understand how Ibuprofen works to treat headaches. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by inhibiting the production of prostaglandins, which are hormone-like substances that cause pain and inflammation. In the case of a headache, the blood vessels in the head dilate, causing pain. Ibuprofen reduces the production of prostaglandins, which in turn reduces inflammation and helps to narrow the blood vessels, relieving the headache.',
    answer='Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID) that works by inhibiting the production of prostaglandins, hormone-like substances that cause pain and inflammation. By reducing the production of prostaglandins, Ibuprofen helps to narrow the blood vessels, relieving headache pain.',
    enough_context='No. The context provided does not mention anything about how Ibuprofen works to treat headaches.'
)